## Backtesting NVDA Options via Monte Carlo & Statistical Arbitrage on Historical Data

In the project we will perform Monte Carlo simulations on historical NVDA options data to see if we can make a profit on mispriced options. 

Some assumptions in this algorithm:

1. No new options are added
2. We only buy the options since this is historical data
3. The stock prices follow the black scholes model 

# NOTE: THIS IS NOT FINANCIAL ADVICE. I AM NOT A FINANCIAL ADVISOR. 

### 1. Data cleaning + transformations




Import libraries


In [1]:
import pandas as pd #linalg
import numpy as np #math 
from scipy.stats import norm #stats for mc sim
from datetime import datetime #datetime library 


import CBOE options data

In [2]:
jan_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-01.csv")
feb_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-02.csv")
mar_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-03.csv")
apr_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-04.csv")
may_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-05.csv")
jun_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-06.csv")
jul_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-07.csv")
aug_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-08.csv")
sep_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-09.csv")
oct_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-10.csv")
nov_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-11.csv")
dec_data = pd.read_csv("CBOE Data/NVDA/UnderlyingOptionsEODCalcs_2023-12.csv")

ls = [jan_data, feb_data, mar_data, apr_data, may_data, jun_data, jul_data, aug_data, sep_data, oct_data, nov_data, dec_data]
data = pd.concat(ls, ignore_index=True)


In [3]:
data.head()

,underlying_symbol,quote_date,root,expiration,strike,option_type,open,high,low,close,...,rho_1545,bid_size_eod,bid_eod,ask_size_eod,ask_eod,underlying_bid_eod,underlying_ask_eod,vwap,open_interest,delivery_code
0,NVDA,2023-01-03,NVDA,2023-01-06,65.0,C,76.85,76.90,76.85,76.90,...,0.0060,25,77.35,25,79.00,143.15,143.16,76.875,2,NaN
1,NVDA,2023-01-03,NVDA,2023-01-06,65.0,P,0.00,0.00,0.00,0.00,...,-0.0000,0,0.00,6,0.01,143.15,143.16,0.000,36,NaN
2,NVDA,2023-01-03,NVDA,2023-01-06,70.0,C,78.40,78.40,78.35,78.35,...,0.0063,25,72.35,25,74.05,143.15,143.16,78.375,2,NaN
3,NVDA,2023-01-03,NVDA,2023-01-06,70.0,P,0.01,0.01,0.01,0.01,...,-0.0000,0,0.00,6,0.01,143.15,143.16,0.010,2,NaN
4,NVDA,2023-01-03,NVDA,2023-01-06,75.0,C,0.00,0.00,0.00,0.00,...,0.0067,25,67.35,25,69.05,143.15,143.16,0.000,40,NaN


options have no id, define function to generate id based on symbol, expdate, strike price, and type

In [5]:
def generate_option_id(option):
    return f"{option['underlying_symbol']}_{option['expiration']}_{option['strike']}_{option['option_type']}"

In [6]:
option_id_series = data.apply(lambda row: generate_option_id(row), axis=1)
data.insert(0, 'option_id', option_id_series)

In [7]:
type(data['expiration'][0])

str

include an interest rate column

In [8]:
data.insert(data.shape[1], 'interest_rate',"")

In [9]:
data.head()

,option_id,underlying_symbol,quote_date,root,expiration,strike,option_type,open,high,low,...,bid_size_eod,bid_eod,ask_size_eod,ask_eod,underlying_bid_eod,underlying_ask_eod,vwap,open_interest,delivery_code,interest_rate
0,NVDA_2023-01-06_65.0_C,NVDA,2023-01-03,NVDA,2023-01-06,65.0,C,76.85,76.90,76.85,...,25,77.35,25,79.00,143.15,143.16,76.875,2,NaN,
1,NVDA_2023-01-06_65.0_P,NVDA,2023-01-03,NVDA,2023-01-06,65.0,P,0.00,0.00,0.00,...,0,0.00,6,0.01,143.15,143.16,0.000,36,NaN,
2,NVDA_2023-01-06_70.0_C,NVDA,2023-01-03,NVDA,2023-01-06,70.0,C,78.40,78.40,78.35,...,25,72.35,25,74.05,143.15,143.16,78.375,2,NaN,
3,NVDA_2023-01-06_70.0_P,NVDA,2023-01-03,NVDA,2023-01-06,70.0,P,0.01,0.01,0.01,...,0,0.00,6,0.01,143.15,143.16,0.010,2,NaN,
4,NVDA_2023-01-06_75.0_C,NVDA,2023-01-03,NVDA,2023-01-06,75.0,C,0.00,0.00,0.00,...,25,67.35,25,69.05,143.15,143.16,0.000,40,NaN,


finding interest rate data

In [10]:
import requests
import fredapi as fd

In [11]:
api_key = 'xxxxxxxxxxx'
url = 'https://api.stlouisfed.org/fred/category?category_id=125&api_key=' + api_key +'&file_type=json'
response = requests.get(url).json()
response

{'categories': [{'id': 125, 'name': 'Trade Balance', 'parent_id': 13}]}

In [12]:
fred = fd.Fred(api_key = api_key)
rate_data = fred.search('Federal Funds Effective Rate')
rate_data.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
FEDFUNDS,FEDFUNDS,2024-03-18,2024-03-18,Federal Funds Effective Rate,1954-07-01,2024-02-01,Monthly,M,Percent,%,Not Seasonally Adjusted,NSA,2024-03-01 15:18:02-06:00,98,Averages of daily figures. For additional hi...
DFF,DFF,2024-03-18,2024-03-18,Federal Funds Effective Rate,1954-07-01,2024-03-15,"Daily, 7-Day",D,Percent,%,Not Seasonally Adjusted,NSA,2024-03-18 15:18:06-05:00,86,For additional historical federal funds rate d...
FF,FF,2024-03-18,2024-03-18,Federal Funds Effective Rate,1954-07-07,2024-03-13,"Weekly, Ending Wednesday",W,Percent,%,Not Seasonally Adjusted,NSA,2024-03-14 15:25:18-05:00,63,Averages of daily figures. For additional hi...
RIFSPFFNB,RIFSPFFNB,2024-03-18,2024-03-18,Federal Funds Effective Rate,1954-07-01,2024-03-15,Daily,D,Percent,%,Not Seasonally Adjusted,NSA,2024-03-18 15:18:22-05:00,44,For additional historical federal funds rate d...
RIFSPFFNA,RIFSPFFNA,2024-03-18,2024-03-18,Federal Funds Effective Rate,1955-01-01,2023-01-01,Annual,A,Percent,%,Not Seasonally Adjusted,NSA,2024-01-02 15:26:02-06:00,42,Averages of daily figures. For additional hi...


We use the DFF series which contains the daily effective rates

In [13]:
rate = fred.get_series('DFF').to_frame()

In [14]:
rate

,0
1954-07-01,1.13
1954-07-02,1.25
1954-07-03,1.25
1954-07-04,1.25
1954-07-05,0.88
...,...
2024-03-11,5.33
2024-03-12,5.33
2024-03-13,5.33
2024-03-14,5.33


In [15]:
year_rate = rate['2023-01-03':'2023-12-29']
year_rate = year_rate.reset_index()
year_rate.rename(columns={'index': 'date'}, inplace=True)
year_rate.rename(columns={0: 'interest_rate'}, inplace=True)
year_rate

,date,interest_rate
0,2023-01-03,4.33
1,2023-01-04,4.33
2,2023-01-05,4.33
3,2023-01-06,4.33
4,2023-01-07,4.33
...,...,...
356,2023-12-25,5.33
357,2023-12-26,5.33
358,2023-12-27,5.33
359,2023-12-28,5.33


In [16]:
year_rate['date'] = pd.to_datetime(year_rate['date']).dt.date
data['quote_date'] = pd.to_datetime(data['quote_date']).dt.date

In [17]:
year_rate.set_index('date', inplace=True)

In [18]:
data['interest_rate'] = data['quote_date'].map(year_rate['interest_rate'])

In [19]:
data

,option_id,underlying_symbol,quote_date,root,expiration,strike,option_type,open,high,low,...,bid_size_eod,bid_eod,ask_size_eod,ask_eod,underlying_bid_eod,underlying_ask_eod,vwap,open_interest,delivery_code,interest_rate
0,NVDA_2023-01-06_65.0_C,NVDA,2023-01-03,NVDA,2023-01-06,65.0,C,76.85,76.90,76.85,...,25,77.35,25,79.00,143.15,143.16,76.8750,2,NaN,4.33
1,NVDA_2023-01-06_65.0_P,NVDA,2023-01-03,NVDA,2023-01-06,65.0,P,0.00,0.00,0.00,...,0,0.00,6,0.01,143.15,143.16,0.0000,36,NaN,4.33
2,NVDA_2023-01-06_70.0_C,NVDA,2023-01-03,NVDA,2023-01-06,70.0,C,78.40,78.40,78.35,...,25,72.35,25,74.05,143.15,143.16,78.3750,2,NaN,4.33
3,NVDA_2023-01-06_70.0_P,NVDA,2023-01-03,NVDA,2023-01-06,70.0,P,0.01,0.01,0.01,...,0,0.00,6,0.01,143.15,143.16,0.0100,2,NaN,4.33
4,NVDA_2023-01-06_75.0_C,NVDA,2023-01-03,NVDA,2023-01-06,75.0,C,0.00,0.00,0.00,...,25,67.35,25,69.05,143.15,143.16,0.0000,40,NaN,4.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040997,NVDA_2026-06-18_910.0_P,NVDA,2023-12-29,NVDA,2026-06-18,910.0,P,0.00,0.00,0.00,...,23,412.00,23,419.90,495.17,495.26,0.0000,0,NaN,5.33
1040998,NVDA_2026-06-18_920.0_C,NVDA,2023-12-29,NVDA,2026-06-18,920.0,C,45.33,45.48,45.33,...,184,45.55,60,47.20,495.17,495.26,45.4529,35,NaN,5.33
1040999,NVDA_2026-06-18_920.0_P,NVDA,2023-12-29,NVDA,2026-06-18,920.0,P,0.00,0.00,0.00,...,30,421.35,22,429.30,495.17,495.26,0.0000,0,NaN,5.33
1041000,NVDA_2026-06-18_930.0_C,NVDA,2023-12-29,NVDA,2026-06-18,930.0,C,44.40,46.21,43.43,...,219,44.25,140,46.15,495.17,495.26,44.5626,38,NaN,5.33


In [20]:
data.to_csv('options_data.csv', index=False)

Run cell below for "data" dataframe without running lines above

In [21]:
data = pd.read_csv("options_data.csv")

In [24]:
data

,option_id,underlying_symbol,quote_date,root,expiration,strike,option_type,open,high,low,...,bid_size_eod,bid_eod,ask_size_eod,ask_eod,underlying_bid_eod,underlying_ask_eod,vwap,open_interest,delivery_code,interest_rate
0,NVDA_2023-01-06_65.0_C,NVDA,2023-01-03,NVDA,2023-01-06,65.0,C,76.85,76.90,76.85,...,25,77.35,25,79.00,143.15,143.16,76.8750,2,NaN,4.33
1,NVDA_2023-01-06_65.0_P,NVDA,2023-01-03,NVDA,2023-01-06,65.0,P,0.00,0.00,0.00,...,0,0.00,6,0.01,143.15,143.16,0.0000,36,NaN,4.33
2,NVDA_2023-01-06_70.0_C,NVDA,2023-01-03,NVDA,2023-01-06,70.0,C,78.40,78.40,78.35,...,25,72.35,25,74.05,143.15,143.16,78.3750,2,NaN,4.33
3,NVDA_2023-01-06_70.0_P,NVDA,2023-01-03,NVDA,2023-01-06,70.0,P,0.01,0.01,0.01,...,0,0.00,6,0.01,143.15,143.16,0.0100,2,NaN,4.33
4,NVDA_2023-01-06_75.0_C,NVDA,2023-01-03,NVDA,2023-01-06,75.0,C,0.00,0.00,0.00,...,25,67.35,25,69.05,143.15,143.16,0.0000,40,NaN,4.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040997,NVDA_2026-06-18_910.0_P,NVDA,2023-12-29,NVDA,2026-06-18,910.0,P,0.00,0.00,0.00,...,23,412.00,23,419.90,495.17,495.26,0.0000,0,NaN,5.33
1040998,NVDA_2026-06-18_920.0_C,NVDA,2023-12-29,NVDA,2026-06-18,920.0,C,45.33,45.48,45.33,...,184,45.55,60,47.20,495.17,495.26,45.4529,35,NaN,5.33
1040999,NVDA_2026-06-18_920.0_P,NVDA,2023-12-29,NVDA,2026-06-18,920.0,P,0.00,0.00,0.00,...,30,421.35,22,429.30,495.17,495.26,0.0000,0,NaN,5.33
1041000,NVDA_2026-06-18_930.0_C,NVDA,2023-12-29,NVDA,2026-06-18,930.0,C,44.40,46.21,43.43,...,219,44.25,140,46.15,495.17,495.26,44.5626,38,NaN,5.33


In [25]:
data['quote_date'] = pd.to_datetime(data['quote_date'])

In [26]:
data['expiration'] = pd.to_datetime(data['expiration'])

## 2. Define Portfolio class

In [28]:
class Portfolio:
    def __init__(self, money):
        self.money = money
        self.positions = pd.DataFrame(columns=['option_id', 'underlying_symbol', 'option_type', 'quantity', 'strike', 'expiration', 'price', 'stock_price']) 
        self.purchased_options = []
        self.owned_stocks = pd.DataFrame(columns=['symbol', 'quantity', 'type', 'stock_price'])
            
    def buy_option(self, data, optionid, symbol, opt_type, quantity, strike, expiration, price, stock_price):
        if self.money <= 0:
            return
        if quantity == 0 or price == 0:
            return
        try:
            quantity = int(quantity)
            price = float(price)
            cost = np.round((price * quantity), 3)
        except ValueError:
            print("invalid quantity or price.")
            return
        
        if self.money >= cost:
            # instantiate newly purchased option
            purchased_option = PurchasedOption(optionid, symbol, opt_type, quantity, strike, expiration, price, stock_price)
            self.purchased_options.append(purchased_option)
            # update capital
            self.money -= cost
            new_position = pd.DataFrame({
                'option_id': [purchased_option.option_id],
                'underlying_symbol': [purchased_option.underlying_symbol],
                'option_type': [purchased_option.option_type],
                'quantity': [purchased_option.quantity],
                'strike': [purchased_option.strike],
                'expiration': [purchased_option.expiration],
                'unit_price': [purchased_option.price],
                'total_cost': [cost],
                'stock_price': [purchased_option.stock_price]  
            })
            # append newly purchased options to positions 
            self.positions = self.positions.append(new_position, ignore_index=True)
            
            # remove options in the future by number of options bought if purchased
            data = data[~((data['option_id'] == optionid))]
            
            # hedge the bought options
            self.hedge_option(data, optionid, symbol, opt_type, quantity, strike, expiration, price)
    
    
    def perform_monte_carlo_simulation (self, optionid, symbol, opt_type, quantity, strike, expiration, price):
        """
        Looks at each option listed on the market, performs N = 10000 monte carlo simulations 
        with the implied volatility associated to it. 
    
        Returns estimated price of the option. 
        """
        N=10_000 
        for index, row in data.iterrows():
            S0=row['underlying_ask_1545']
            r=row['interest_rate']
            expiration_date = row['expiration']
            T = (expiration_date - datetime.now()).days / 365.0
            K=row['strike'];
            sigma=row['implied_volatility_1545']; #saves us solving the black scholes model 
        #calculate payoffs 
        payoff = np.zeros(N)
        for i in range(N):
            # random number generation 
            WT = np.sqrt(T) * np.random.randn()  # using np.random.randn() for normal distribution
            ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * WT)
            payoff[i] = np.exp(-r * T) * max(ST - K, 0)
            
        MC_option_price = np.mean(payoff)
        MC_error=3*np.sqrt(np.var(payoff))/np.sqrt(N)
        
        array = [MC_option_price,MC_error]
        
        return array 

    def solve_binomial_model(self, data, optionid, symbol, opt_type, quantity, strike, expiration, price):
        N = 5
        dt = row['expiration'] / N
        q = (np.exp(r * dt) - d) / (u - d)
        disc = np.exp(-r * dt)

        # Initialise stock prices at maturity
        S = S0 * d**(np.arange(N, -1, -1)) * u**(np.arange(0, N + 1, 1))

        # Option payoff at maturity
        if opttype == 'P':
            C = np.maximum(0, K - S)
        else:
            C = np.maximum(0, S - K)

        # Backward recursion through the tree
        for i in np.arange(N - 1, -1, -1):
            S = S0 * d**(np.arange(i, -1, -1)) * u**(np.arange(0, i + 1, 1))
            C[:i + 1] = disc * (q * C[1:i + 2] + (1 - q) * C[0:i + 1])

            # Stop after the first step and return the option price today, and prices for up and down movements
            if i == N - 1:
                option_price_today = C[0]
                option_price_up = C[1]
                option_price_down = C[0]
                return option_price_today, option_price_up, option_price_down

            C = C[:-1]
            if opttype == 'P':
                C = np.maximum(C, K - S)
            else:
                C = np.maximum(C, S - K)

        return C[0]
          
    
    # For each option I own, I perform a continuation value calculation 
    def solve_continuation_value(self, optionid, symbol, opt_type, quantity, strike, expiration, price):
        # Get the current and next step option prices
        option_price_today, option_price_up, option_price_down = self.solve_binomial_model(data, optionid, symbol, opt_type, quantity, strike, expiration, price)

        # Calculate parameters for continuation value calculation
        N = 10  # Number of steps
        T = (expiration - datetime.now()).days / 365.0
        dt = row['expiration'] / N
        r = data.loc[data['option_id'] == optionid, 'interest_rate'].values[0]
        sigma = data.loc[data['option_id'] == optionid, 'implied_volatility_1545'].values[0]
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        q_u = (np.exp(r * dt) - d) / (u - d)
        disc = np.exp(-r * dt)

        # Calculate continuation value
        continuation_value = disc * (q_u * option_price_up + (1 - q_u) * option_price_down)

        return continuation_value

  
    # iterates through my positions to see if they should be exercised; calls "solve_continuation_value" each time
    def exercise_option(self, data, optionid, symbol, opt_type, quantity, strike, expiration, price):
        for index, row in data.iterrows():
            current_value = self.solve_binomial_model(data, row['option_id'], row['underlying_symbol'], row['option_type'], row['quantity'], row['strike'], row['expiration'], row['price'])
            continuation_value = self.solve_continuation_value(data, row['option_id'], row['underlying_symbol'], row['option_type'], row['quantity'], row['strike'], row['expiration'], row['price'])
            
            if current_value > continuation_value:
                self.settle_option(row['option_id'], row['underlying_symbol'], row['option_type'], row['quantity'], row['strike'], row['expiration'], row['price'])
            
            
    def settle_option(self, option_id, symbol, opt_type, quantity, strike, expiration, price):
        if opt_type == 'C':  # exercising a call option
            # Buy the underlying asset at the strike price
            cost = strike * quantity
            if self.money >= cost:
                self.money -= cost  # Deduct the cost from the portfolio's cash
                # Add the underlying asset to owned stocks
                self.owned_stocks = self.owned_stocks.append({'symbol': symbol, 'quantity': quantity, 'type': 'long', 'stock_price': strike}, ignore_index=True)

        elif opt_type == 'P':
            # Check if there is stock to sell
            asset_row = self.owned_stocks[(self.owned_stocks['symbol'] == symbol) & (self.owned_stocks['type'] == 'long')]
            if not asset_row.empty and asset_row.iloc[0]['quantity'] >= quantity:
                self.money += strike * quantity  # Receive the money from selling the asset
                self.owned_stocks.at[asset_row.index[0], 'quantity'] -= quantity  # Deduct the asset from owned stocks
                # If the holding goes to zero, remove it from the DataFrame
                if self.owned_stocks.at[asset_row.index[0], 'quantity'] == 0:
                    self.owned_stocks = self.owned_stocks.drop(asset_row.index)

        # remove the exercised option from the portfolio
        self.positions = self.positions[self.positions['option_id'] != option_id]

     
            # remove the exercised option from the portfolio
        self.positions = self.positions[self.positions['option_id'] != option_id]
        
    def hedge_option(self, data, option_id, symbol, opt_type, quantity, strike, expiration, price):
        # Calculate parameters for the binomial model
        N = 1  # Since we are only looking at tomorrow
        T = (expiration - datetime.now()).days / 365.0
        dt = row['expiration'] / N
        r = row['interest_rate']
        sigma = row['implied_volatility_1545']
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        S0 = row['underlying_ask_1545']

        # Get option prices for the next step (up and down) using the binomial model
        _, C_u, C_d = self.solve_binomial_model(data, option_id, symbol, opt_type, quantity, strike, expiration, price)

        # Calculate stock prices after an up and down movement
        S_u = S0 * u
        S_d = S0 * d

        # Calculate delta
        delta = (C_u - C_d) / (S_u - S_d)

        # Determine the number of stocks to buy or short-sell for hedging
        stocks_to_hedge = delta * quantity

        if delta < 0:
            # Short stock and add it to portfolio
            self.money += -stocks_to_hedge * price
            self.owned_stocks = self.owned_stocks.append({'symbol': symbol, 'quantity': -stocks_to_hedge, 'type': 'short'}, ignore_index=True)
        elif delta > 0:
            # Buy stock and add it to portfolio
            self.money -= stocks_to_hedge * price
            self.owned_stocks = self.owned_stocks.append({'symbol': symbol, 'quantity': stocks_to_hedge, 'type': 'long'}, ignore_index=True)
        else:  # if delta is 0
            # do nothing
            pass
    
    def close_option(self, option_id, symbol, opt_type, quantity, strike, expiration, price):
        # Close the option position
        current_stock_price = row['underlying_ask_1545']
        
        if opt_type == 'C' and current_stock_price > row['strike']:
            # Call option is in the money, exercise it
            self.settle_option(option_id, symbol, opt_type, quantity, strike, expiration, price)
            print(f"Exercised call option {option_id}.")
            
        elif opt_type == 'P' and current_stock_price < strike:
            # Put option is in the money, exercise it
            self.settle_option(option_id, symbol, opt_type, quantity, strike, expiration, price)
            print(f"Exercised put option {option_id}.")
            
        else:
            #Option is out of the money, let it expire and do nothing
            pass

        # Remove the option from the portfolio, whether it was exercised or expired
        self.positions = self.positions[self.positions['option_id'] != option_id]
        
    
    # at the end of the data, liquidate everything 
    def liquidate_portfolio(self):
        # Liquidate all option positions
        for index, row in self.positions.iterrows():
            self.close_option(row, row['option_id'], row['underlying_symbol'], row['option_type'], row['quantity'], row['strike'], row['expiration'], row['price'])
        
        # Liquidate all stock positions
        for index, row in self.owned_stocks.iterrows():
            stock_quantity = row['quantity']
            stock_price = row['underlying_ask_1545']
            if row['type'] == 'long':
                # Sell the long position
                self.money += stock_quantity * stock_price
            elif row['type'] == 'short':
                # Buy back the short position
                self.money -= stock_quantity * stock_price

class PurchasedOption:
    def __init__(self, optionid, symbol, opt_type, quantity, strike, expiration, price,stock_price):
        self.option_id = optionid
        self.underlying_symbol = symbol
        self.option_type = opt_type
        self.quantity = quantity
        self.strike = strike 
        self.expiration = expiration 
        self.price = price 
        self.stock_price = stock_price  
        
        
        
        


## 3. Backtesting 

In general: 

- Go through each row of the data, use the parameters
- Set current date to row['quote_date']
- Go through the data until the for loop hits a row['quote_date'] that is different than what it currently is defined (means that 1 day has passed) 

In [ ]:
my_port = Portfolio(1_000_000)  # give myself 1 million dollars

current_day = None  #current day

temp = data[0:100]
for index, row in temp.iterrows():
    if current_day is None:
        current_day = row['quote_date']  # Set the start date

    # Perform Monte Carlo simulation
    array = my_port.perform_monte_carlo_simulation(row['option_id'], 
                                                   row['underlying_symbol'],
                                                   row['option_type'],
                                                   row['bid_size_eod'],
                                                   row['strike'],
                                                   row['expiration'],
                                                   row['bid_eod'])
    
    MC_option_price, MC_error = array
    
    # Buy underpriced call options
    if row['option_type'] == 'C' and row['bid_eod'] < MC_option_price - MC_error:
        my_port.buy_option(row, 
                           row['option_id'], 
                           row['underlying_symbol'],
                           row['option_type'],
                           row['bid_size_eod'],
                           row['strike'],
                           row['expiration'],
                           row['bid_eod'],
                           row['underlying_ask_1545'])
                          

    # Buy overpriced put options 
    elif row['option_type'] == 'P' and row['bid_eod'] > MC_option_price + MC_error:
        my_port.buy_option(row,
                           row['option_id'], 
                           row['underlying_symbol'],
                           row['option_type'],
                           row['bid_size_eod'],
                           row['strike'],
                           row['expiration'],
                           row['bid_eod'],
                           row['underlying_ask_1545'])
                          

    # check  if the day has ended
    if index < len(data) - 1 and current_day != data.iloc[index + 1]['quote_date']:
        for idx, position_row in my_port.positions.iterrows():
            current_market_price = position_row['underlying_ask_1545']  

            # exercise call if they are in the money
            if position_row['option_type'] == 'C' and current_market_price > position_row['strike']:
                my_port.settle_option(position_row['option_id'], position_row['underlying_symbol'], position_row['option_type'], position_row['quantity'], position_row['strike'], position_row['expiration'], position_row['price'])

            # exercise put if they are in the money
            elif position_row['option_type'] == 'P' and current_market_price < position_row['strike']:
                my_port.settle_option(position_row['option_id'], position_row['underlying_symbol'], position_row['option_type'], position_row['quantity'], position_row['strike'], position_row['expiration'], position_row['price'])

        current_day = data.iloc[index + 1]['quote_date']  # Update the day
